### Import Libraries

In [10]:
import pandas as pd
import sqlalchemy
import seaborn as sns
import requests
import matplotlib.pyplot as plt
from nba_api.stats.static import teams
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import teamgamelogs
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamefinder
import time
import sqlite3

# sqlite3 docs
    # https://docs.python.org/3/library/sqlite3.html

### Headers

In [11]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

### Connect to SQLite DB

In [2]:
db = "NBA.db"

In [3]:
conn = sqlite3.connect('NBA.db')

In [4]:
c = conn.cursor()

### Teams Table

In [8]:
teams_list = teams.get_teams()
teams_df = pd.DataFrame(teams_list)

In [14]:
teams_df.to_sql(
    "teams",
    conn,
    if_exists='append',
    index=False)

30

### Team ID Table

In [10]:
team_id_df = teams_df['id'].rename('team_id')

In [13]:
team_id_df.to_sql(
    "team_id",
    conn,
    if_exists='append',
    index=False)

30

### Players (Active) Table

In [17]:
active_players = players.get_active_players()
active_players_df = pd.DataFrame(active_players)
active_players_df = active_players_df.rename(columns={'id':'PLAYER_ID'})
active_players_df

,PLAYER_ID,full_name,first_name,last_name,is_active
0,1630173,Precious Achiuwa,Precious,Achiuwa,True
1,203500,Steven Adams,Steven,Adams,True
2,1628389,Bam Adebayo,Bam,Adebayo,True
3,1630534,Ochai Agbaji,Ochai,Agbaji,True
4,1630583,Santi Aldama,Santi,Aldama,True
...,...,...,...,...,...
577,201152,Thaddeus Young,Thaddeus,Young,True
578,1629027,Trae Young,Trae,Young,True
579,1630209,Omer Yurtseven,Omer,Yurtseven,True
580,203469,Cody Zeller,Cody,Zeller,True


In [18]:
active_players_df.to_sql(
    "active_players",
    conn,
    if_exists='append',
    index=False)

582

### Player ID table

In [19]:
active_player_ids_df = active_players_df['PLAYER_ID']
active_player_ids_df

0      1630173
1       203500
2      1628389
3      1630534
4      1630583
        ...   
577     201152
578    1629027
579    1630209
580     203469
581    1627826
Name: PLAYER_ID, Length: 582, dtype: int64

In [22]:
active_player_ids_df.to_sql(
    "active_player_id",
    conn,
    if_exists='append',
    index=False)

582

### Player Info Table

In [20]:
#function that gets player info data
def get_player_data(active_player_ids):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=active_player_ids, headers=headers, timeout=30)
    player_info_df = player_info.common_player_info.get_data_frame()
    return player_info_df

In [21]:
player_data = []
for nba_player_ids in active_player_ids_df:
    print(nba_player_ids)
    time.sleep(3.1)
    player_info = get_player_data(nba_player_ids)
    player_data.append(player_info)
player_data_df = pd.concat(player_data, ignore_index=True)

1630173
203500
1628389
1630534
1630583
1629638
1628960
1628386
1630631
203937
203507
1628961
203648
1630175
1628384
1627853
1630166
1629028
1628962
1628963
1631116
1630163
1629071
1628366
1628964
1631094
1630217
1630625
1631230
203084
1630567
1629628
203115
1629646
1628966
201587
203145
1629647
203078
1627736
1630699
1631262
202722
201976
1630180
1631234
203920
1629048
202687
1631205
203992
202711
1629626
1629716
1630195
1626164
1630527
1628449
1630547
1631123
1628396
1631103
1631128
1628969
1629052
1627763
1628415
1628971
1631122
1627759
1631112
1629650
1630535
1629718
1628972
1628973
1628418
203493
203504
202692
1630215
202710
1631288
203484
1628427
1629962
203991
1630176
1628975
1628976
1627936
1626161
1630577
1630551
1631113
1629597
1629185
1627737
1631108
1630528
1629634
203903
1629651
1629599
1628381
1628380
201144
1626192
1626188
1630536
203496
1628470
203109
1630622
1629633
1630595
203552
201939
1630700
1630268
203076
1631098
1629056
1631120
1630620
201942
203473
203521
1628978

In [24]:
player_data_df.to_sql(
    "active_player_data",
    conn,
    if_exists='append',
    index=False)

582

### Game Results Table (total by team)

In [13]:
def get_game_results(season):
    game_results = leaguegamefinder.LeagueGameFinder(season_nullable=season,league_id_nullable='00',season_type_nullable='Regular Season')
    game_results_df = game_results.get_data_frames()[0]
    return game_results_df

In [20]:
# seasons = ['2010-11','2011-12','2012-13','2013-14','2014-15','2015-16','2016-17','2017-18','2018-19','2019-20','2020-21','2021-22','2022-23']
# seasons = ['2022-23']
seasons = ['2010-11']

In [21]:
game_data = []
for season in seasons:
    print(season)
    game_results = get_game_results(season)
    game_data.append(game_results)
    # player_info_df = player_info.common_player_info.get_data_frame()
    # time.sleep(3)
all_game_results_df = pd.concat(game_data, ignore_index=True)

2010-11


In [22]:
all_game_results_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22010,1610612751,NJN,New Jersey Nets,0021001222,2011-04-13,NJN @ CHI,L,240,92,...,0.875,8,27,35,21,6,6,11,19,-5.0
1,22010,1610612766,CHA,Charlotte Bobcats,0021001217,2011-04-13,CHA vs. ATL,W,240,96,...,0.813,7,30,37,27,7,7,8,16,11.0
2,22010,1610612755,PHI,Philadelphia 76ers,0021001220,2011-04-13,PHI vs. DET,L,241,100,...,0.875,9,31,40,27,6,6,8,22,-4.0
3,22010,1610612760,OKC,Oklahoma City Thunder,0021001225,2011-04-13,OKC vs. MIL,L,263,106,...,0.778,13,39,52,26,9,7,23,20,-4.0
4,22010,1610612745,HOU,Houston Rockets,0021001224,2011-04-13,HOU @ MIN,W,239,121,...,0.781,12,36,48,32,5,3,15,18,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22010,1610612747,LAL,Los Angeles Lakers,0021000003,2010-10-26,LAL vs. HOU,W,240,112,...,0.821,14,30,44,21,11,4,12,24,2.0
2456,22010,1610612748,MIA,Miami Heat,0021000001,2010-10-26,MIA @ BOS,L,242,80,...,0.720,11,28,39,15,10,6,17,21,-8.0
2457,22010,1610612757,POR,Portland Trail Blazers,0021000002,2010-10-26,POR vs. PHX,W,240,106,...,0.667,18,30,48,31,11,2,12,22,14.0
2458,22010,1610612738,BOS,Boston Celtics,0021000001,2010-10-26,BOS vs. MIA,W,239,88,...,0.640,8,34,42,25,6,4,18,19,8.0


In [23]:
all_game_results_df.to_sql(
    "game_results",
    conn,
    if_exists='append',
    index=False)

NameError: name 'conn' is not defined

### Game ID Table

In [24]:
game_id_df = all_game_results_df['GAME_ID'].unique()
game_id_df = pd.DataFrame(game_id_df)
game_id_df = game_id_df.rename(columns={0:'GAME_ID'})
game_id_df

,GAME_ID
0,0021001222
1,0021001217
2,0021001220
3,0021001225
4,0021001224
...,...
1225,0021000011
1226,0021000006
1227,0021000003
1228,0021000001


In [33]:
game_id_df.to_sql(
    "game_id",
    conn,
    if_exists='append',
    index=False)

386

### Boxscore Table

In [25]:
#create function that gets box score data based on game_ids
def get_box_score_data(game_id):
    player_stat_data = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id, headers=headers, timeout = 30) # add headers=headers if errors occur, it was causing error for me though
    Player_stat_data_df = player_stat_data.player_stats.get_data_frame()
    return Player_stat_data_df

In [26]:
boxscores = []

for game_id in game_id_df.values.tolist():
    print(game_id)
    time.sleep(3.0)
    boxscore_data = get_box_score_data(game_id)
    boxscores.append(boxscore_data)

all_boxscores_df = pd.concat(boxscores, ignore_index=True)

['0021001222']
['0021001217']


KeyboardInterrupt: 

In [37]:
all_boxscores_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022200378,1610612764,WAS,Washington,1630166,Deni Avdija,Deni,F,,39:53,...,2.0,7.0,9.0,4.0,1.0,0.0,3.0,4.0,14.0,-5.0
1,0022200378,1610612764,WAS,Washington,1628398,Kyle Kuzma,Kyle,F,,35:37,...,2.0,5.0,7.0,7.0,0.0,0.0,3.0,2.0,27.0,-13.0
2,0022200378,1610612764,WAS,Washington,204001,Kristaps Porzingis,Kristaps,C,,34:39,...,2.0,7.0,9.0,2.0,0.0,2.0,4.0,1.0,29.0,-7.0
3,0022200378,1610612764,WAS,Washington,1630557,Corey Kispert,Corey,G,,32:01,...,1.0,3.0,4.0,2.0,0.0,0.0,1.0,4.0,8.0,1.0
4,0022200378,1610612764,WAS,Washington,1628420,Monte Morris,Monte,G,,34:30,...,0.0,2.0,2.0,5.0,0.0,0.0,1.0,3.0,7.0,-13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9912,0022200001,1610612738,BOS,Boston,1631120,JD Davison,JD,,DNP - Coach's Decision,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9913,0022200001,1610612738,BOS,Boston,1628382,Justin Jackson,Justin,,DNP - Coach's Decision,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9914,0022200001,1610612738,BOS,Boston,1629662,Mfiondu Kabengele,Mfiondu,,DNP - Coach's Decision,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9915,0022200001,1610612738,BOS,Boston,1628436,Luke Kornet,Luke,,DNP - Coach's Decision,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
all_boxscores_df.to_sql(
    "boxscores",
    conn,
    if_exists='append',
    index=False)

9917

### Schedules Table (from csv)

In [7]:
schedules_df = pd.read_csv("NBA_Schedule_2022.csv")
schedules_df

,game_id,sequence,season,game_date,game_date_time,away,home,matchup,arena,city,state
0,12200001,1,2022-23,2022-09-30,2022-09-30T06:00:00,WAS,GSW,GSW @ WAS,Saitama Super Arena,Tokyo,NaN
1,12200002,2,2022-23,2022-09-30,2022-09-30T22:00:00,LAC,MRA,MRA @ LAC,Climate Pledge Arena,Seattle,WA
2,12200003,1,2022-23,2022-10-01,2022-10-01T20:00:00,MIL,MEM,MEM @ MIL,Fiserv Forum,Milwaukee,WI
3,12200004,1,2022-23,2022-10-02,2022-10-02T01:00:00,GSW,WAS,WAS @ GSW,Saitama Super Arena,Tokyo,NaN
4,12200005,2,2022-23,2022-10-02,2022-10-02T13:00:00,BOS,CHA,CHA @ BOS,TD Garden,Boston,MA
...,...,...,...,...,...,...,...,...,...,...,...
1295,22201226,11,2022-23,2023-04-09,2023-04-09T15:30:00,OKC,MEM,MEM @ OKC,Paycom Center,Oklahoma City,OK
1296,22201227,12,2022-23,2023-04-09,2023-04-09T15:30:00,DEN,SAC,SAC @ DEN,Ball Arena,Denver,CO
1297,22201228,13,2022-23,2023-04-09,2023-04-09T15:30:00,LAL,UTA,UTA @ LAL,Crypto.com Arena,Los Angeles,CA
1298,22201229,14,2022-23,2023-04-09,2023-04-09T15:30:00,PHX,LAC,LAC @ PHX,Footprint Center,Phoenix,AZ


In [9]:
schedules_df.to_sql(
    "schedule",
    conn,
    if_exists='append',
    index=False)

1300

### Conference Table (from CSV)

In [19]:
conferences_df = pd.read_csv("NBA_Conferences.csv", encoding= 'unicode_escape')
conferences_df

,team_name,team_acronym,conference,division,logo
0,Toronto Raptors,TOR,Eastern,Atlantic,http://content.sportslogos.net/logos/6/227/thu...
1,Boston Celtics,BOS,Eastern,Atlantic,http://content.sportslogos.net/logos/6/213/thu...
2,Philadelphia 76ers,PHI,Eastern,Atlantic,http://content.sportslogos.net/logos/6/218/thu...
3,Brooklyn Nets,BKN,Eastern,Atlantic,http://content.sportslogos.net/logos/6/3786/th...
4,New York Knicks,NYK,Eastern,Atlantic,http://content.sportslogos.net/logos/6/216/thu...
5,Milwaukee Bucks,MIL,Eastern,Central,http://content.sportslogos.net/logos/6/225/thu...
6,Indiana Pacers,IND,Eastern,Central,http://content.sportslogos.net/logos/6/224/thu...
7,Detroit Pistons,DET,Eastern,Central,http://content.sportslogos.net/logos/6/223/thu...
8,Chicago Bulls,CHI,Eastern,Central,http://content.sportslogos.net/logos/6/221/thu...
9,Cleveland Cavaliers,CLE,Eastern,Central,http://content.sportslogos.net/logos/6/222/thu...


In [ ]:
conn = sqlite3.connect(db)
conferences_df.to_sql(
    "conferences",
    conn,
    if_exists='append',
    index=False)